In [119]:
# In this scenario we are going to grab the info from excel files.
'''
We need:
1. Prices in different currencies
2. List with Pusblisher names.
3. Clean Publisher Name
4. Check Suppliers, currencies, publishers, discount, shipping costs
5. Check Suppliers special discounts
6. Check restrictions
7. Get landed Cost
8. List of ISBN to ananlyse.

'''

'\nWe need:\n1. Prices in different currencies\n2. List with Pusblisher names.\n3. Clean Publisher Name\n4. Check Suppliers, currencies, publishers, discount, shipping costs\n5. Check Suppliers special discounts\n6. Check restrictions\n7. Get landed Cost\n8. List of ISBN to ananlyse.\n\n'

In [120]:
# 0. Import Libraries and credentials
import pandas as pd
import pyodbc
import numpy as np
import sqlCredentials as sql

In [121]:
# 0. Get databse connection
# Let's pick all the titles gwe can get from the pricing matrix.

prod_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=BookXCenterProduction;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

prouc_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=PROCUREMENTDB;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

In [122]:
#1. Prices at different currencies
#this time we are going to use all the titles
list_price_sql = """
SELECT *
FROM Isbn.ListPrice
"""

list_price = pd.read_sql(list_price_sql, prod_db)
list_price.head()

,Id,Isbn,Currency,Price,PriceSource,PriceLastRefreshed,Timestamp
0,1,9780008135904,AUD,12.99,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-07 15:03:40.623333
1,2,9780008135904,CND,9.99,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-07 15:03:40.623333
2,3,9780008135904,GBP,4.99,BertramsInventory,2020-04-07 13:00:00.0163119,2020-04-07 15:03:40.623333
3,4,9780008135904,NZD,14.99,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-07 15:03:40.623333
4,5,9780008135935,GBP,4.99,BertramsInventory,2020-04-07 13:00:00.0163119,2020-04-07 15:03:40.623333


In [123]:
list_price.shape

(1626132, 7)

In [124]:
list_price['PriceSource'].unique()

array(['BowkerNepModel', 'BertramsInventory', 'GoldenList',
       'BookDepotInventory', 'NielsenFtp', 'TitlePageScraper',
       'BowkerApiModel', 'PearsonCanadaScraperModel', 'SAPPurchaseOrder',
       'AbePolandScraper'], dtype=object)

In [125]:
list_price.drop(columns = ['Id', 'PriceSource', 'PriceLastRefreshed', 'Timestamp'], inplace = True)
list_price.head()

,Isbn,Currency,Price
0,9780008135904,AUD,12.99
1,9780008135904,CND,9.99
2,9780008135904,GBP,4.99
3,9780008135904,NZD,14.99
4,9780008135935,GBP,4.99


In [126]:
bilblo_sql = """
SELECT *
FROM Isbn.Bibliography
"""
biblio = pd.read_sql(bilblo_sql, prod_db)
biblio.head()

,Isbn,Title,TitleSource,TitleLastRefreshed,Publisher,PublisherSource,PublisherLastRefreshed,Author,AuthorSource,AuthorLastRefreshed,...,PublicationDateSource,PublicationDateLastRefreshed,Copyright,CopyrightSource,CopyrightLastRefreshed,Binding,BindingSource,BindingLastRefreshed,Timestamp,Edition
0,9780000002402,None,None,None,W KLUWER,GoldenList,2020-04-07 12:00:00.5366667,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-07 15:02:46.700,None
1,9780000002679,None,None,None,W KLUWER,GoldenList,2020-04-07 12:00:00.5366667,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-07 15:02:46.700,None
2,9780000003102,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-07 15:02:46.700,None
3,9780000003201,None,None,None,W KLUWER,GoldenList,2020-04-07 12:00:00.5366667,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-07 15:02:46.700,None
4,9780000006493,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,2020-04-07 15:02:46.700,None


In [127]:
sup_list = pd.merge(list_price, biblio, how='left', on = 'Isbn' )

In [128]:
sup_list.head()

,Isbn,Currency,Price,Title,TitleSource,TitleLastRefreshed,Publisher,PublisherSource,PublisherLastRefreshed,Author,...,PublicationDateSource,PublicationDateLastRefreshed,Copyright,CopyrightSource,CopyrightLastRefreshed,Binding,BindingSource,BindingLastRefreshed,Timestamp,Edition
0,9780008135904,AUD,12.99,Collins Mandarin Chinese Phrasebook and Dictio...,BertramsInventory,2020-04-07 13:00:00.0163119,None,None,None,None,...,BowkerNepModel,2020-03-06 21:00:00.0081342,2017,BowkerNepModel,2020-03-06 21:00:00.0081342,Paperback,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-07 15:02:46.700,3
1,9780008135904,CND,9.99,Collins Mandarin Chinese Phrasebook and Dictio...,BertramsInventory,2020-04-07 13:00:00.0163119,None,None,None,None,...,BowkerNepModel,2020-03-06 21:00:00.0081342,2017,BowkerNepModel,2020-03-06 21:00:00.0081342,Paperback,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-07 15:02:46.700,3
2,9780008135904,GBP,4.99,Collins Mandarin Chinese Phrasebook and Dictio...,BertramsInventory,2020-04-07 13:00:00.0163119,None,None,None,None,...,BowkerNepModel,2020-03-06 21:00:00.0081342,2017,BowkerNepModel,2020-03-06 21:00:00.0081342,Paperback,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-07 15:02:46.700,3
3,9780008135904,NZD,14.99,Collins Mandarin Chinese Phrasebook and Dictio...,BertramsInventory,2020-04-07 13:00:00.0163119,None,None,None,None,...,BowkerNepModel,2020-03-06 21:00:00.0081342,2017,BowkerNepModel,2020-03-06 21:00:00.0081342,Paperback,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-07 15:02:46.700,3
4,9780008135935,GBP,4.99,Collins Portuguese Phrasebook and Dictionary G...,BertramsInventory,2020-04-07 13:00:00.0163119,None,None,None,None,...,BowkerNepModel,2020-03-06 21:00:00.0081342,2016,BowkerNepModel,2020-03-06 21:00:00.0081342,Paperback,BowkerNepModel,2020-03-06 21:00:00.0081342,2020-04-07 15:02:46.700,4


In [129]:
sup_list.columns

Index(['Isbn', 'Currency', 'Price', 'Title', 'TitleSource',
       'TitleLastRefreshed', 'Publisher', 'PublisherSource',
       'PublisherLastRefreshed', 'Author', 'AuthorSource',
       'AuthorLastRefreshed', 'PublicationDate', 'PublicationDateSource',
       'PublicationDateLastRefreshed', 'Copyright', 'CopyrightSource',
       'CopyrightLastRefreshed', 'Binding', 'BindingSource',
       'BindingLastRefreshed', 'Timestamp', 'Edition'],
      dtype='object')

In [130]:
sup_list = sup_list[['Isbn','Currency', 'Price','Title', 'Publisher', 'Author']]

In [131]:
sup_list.head()

,Isbn,Currency,Price,Title,Publisher,Author
0,9780008135904,AUD,12.99,Collins Mandarin Chinese Phrasebook and Dictio...,None,None
1,9780008135904,CND,9.99,Collins Mandarin Chinese Phrasebook and Dictio...,None,None
2,9780008135904,GBP,4.99,Collins Mandarin Chinese Phrasebook and Dictio...,None,None
3,9780008135904,NZD,14.99,Collins Mandarin Chinese Phrasebook and Dictio...,None,None
4,9780008135935,GBP,4.99,Collins Portuguese Phrasebook and Dictionary G...,None,None


In [132]:
sup_list['Publisher'].unique()

array([None, 'HARPER COLLINS', 'W KLUWER', ...,
       'United Nations Publications',
       'United Nations Fund for Population Activities',
       'World Health Organization'], dtype=object)

In [133]:
sup_list = sup_list.fillna('N/A')
sup_list.isna().sum()

Isbn         0
Currency     0
Price        0
Title        0
Publisher    0
Author       0
dtype: int64

In [134]:
# 3. Clean publisher
dictio_sql = """
SELECT *
FROM dbo.PublisherDictionary
"""

dictio = pd.read_sql(dictio_sql, prouc_db)
dictio.head()

,PublisherLong,PublisherShort
0,"""Abrams, Inc.""",ABRAMS
1,"""Addison-Wesley Longman, Incorporated""",PEARSON
2,"""Addison-Wesley Longman, Limited""",PEARSON
3,"""Allyn & Bacon, Incorporated""",PEARSON
4,"""American Occupational Therapy Association, In...","American Occupational Therapy Association, Inc..."


In [135]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

In [136]:
for col in list(dictio.columns):
    dictio[col] = dictio.apply(lambda x: clean_up(x[col]), axis =1)

In [137]:
dictio.head()

,PublisherLong,PublisherShort
0,"""ABRAMS, INC.""",ABRAMS
1,"""ADDISON-WESLEY LONGMAN, INCORPORATED""",PEARSON
2,"""ADDISON-WESLEY LONGMAN, LIMITED""",PEARSON
3,"""ALLYN & BACON, INCORPORATED""",PEARSON
4,"""AMERICAN OCCUPATIONAL THERAPY ASSOCIATION, IN...","AMERICAN OCCUPATIONAL THERAPY ASSOCIATION, INC..."


In [138]:
for col in list(sup_list.drop(columns = 'Price').columns):
    sup_list[col] = sup_list.apply(lambda x: clean_up(x[col]), axis =1)

In [139]:
sup_list.head()

,Isbn,Currency,Price,Title,Publisher,Author
0,9780008135904,AUD,12.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A
1,9780008135904,CND,9.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A
2,9780008135904,GBP,4.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A
3,9780008135904,NZD,14.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A
4,9780008135935,GBP,4.99,COLLINS PORTUGUESE PHRASEBOOK AND DICTIONARY G...,N/A,N/A


In [140]:
# Merge on Publisher
sup_list2 = pd.merge(sup_list, dictio, how='left', left_on = 'Publisher', right_on = 'PublisherLong' )

In [141]:
sup_list2.head()

,Isbn,Currency,Price,Title,Publisher,Author,PublisherLong,PublisherShort
0,9780008135904,AUD,12.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A,N/A,N/A
1,9780008135904,CND,9.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A,N/A,N/A
2,9780008135904,GBP,4.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A,N/A,N/A
3,9780008135904,NZD,14.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A,N/A,N/A
4,9780008135935,GBP,4.99,COLLINS PORTUGUESE PHRASEBOOK AND DICTIONARY G...,N/A,N/A,N/A,N/A


In [142]:
sup_list2['Currency'].unique()

array(['AUD', 'CND', 'GBP', 'NZD', 'USD', 'EURO', 'PLN'], dtype=object)

In [143]:
# 4. get suppliers discount
sup_pub = pd.read_excel('../../../../Vendors/.  Vendors Details\supplier_procurement_details.xlsx',  sheet_name='pub')
sup_pub.head()

,Supplier,Publisher,Currency,Discount,Shipping Cost,max quantity,Min,Region
0,COINFO,3D TOTAL,AUD,0.22,3.3,10000,0,AUSTRALIA
1,COINFO,ABC CLIO,AUD,0.30,3.3,10000,0,AUSTRALIA
2,COINFO,AIAA,AUD,0.30,3.3,10000,0,AUSTRALIA
3,COINFO,ALIEN & UNWIN,AUD,0.30,3.3,10000,0,AUSTRALIA
4,COINFO,Amer Psych Ass.,AUD,0.33,3.3,10000,0,AUSTRALIA


In [144]:
for col in ['Supplier', 'Publisher', 'Currency', 'Region']:
    sup_pub[col] = sup_pub.apply(lambda x: clean_up(x[col]), axis =1)

In [145]:
sup_list3 = pd.merge(sup_list2, sup_pub, how='inner', left_on = ['Currency', 'PublisherShort'], right_on = ['Currency', 'Publisher'] )

In [146]:
sup_list3

,Isbn,Currency,Price,Title,Publisher_x,Author,PublisherLong,PublisherShort,Supplier,Publisher_y,Discount,Shipping Cost,max quantity,Min,Region
0,9780000079145,GBP,67.00,N/A,W KLUWER,N/A,W KLUWER,W KLUWER,ABE,W KLUWER,0.55,3.47,10000,0,EUROPE
1,9780000002679,GBP,425.00,N/A,W KLUWER,N/A,W KLUWER,W KLUWER,ABE,W KLUWER,0.55,3.47,10000,0,EUROPE
2,9780000050090,GBP,17.99,N/A,W KLUWER,N/A,W KLUWER,W KLUWER,ABE,W KLUWER,0.55,3.47,10000,0,EUROPE
3,9780000003201,GBP,265.00,N/A,W KLUWER,N/A,W KLUWER,W KLUWER,ABE,W KLUWER,0.55,3.47,10000,0,EUROPE
4,9780000002402,GBP,306.00,N/A,W KLUWER,N/A,W KLUWER,W KLUWER,ABE,W KLUWER,0.55,3.47,10000,0,EUROPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493815,9789027252722,AUD,78.95,"PIDGINS, CREOLES AND MIXED LANGUAGES: AN INTRO...",JOHN BENJAMINS PUBLISHING,VIVEKA VELUPILLAI,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA
493816,9789027212092,AUD,79.95,HISTORY OF THE ENGLISH LANGUAGE,JOHN BENJAMINS PUBLISHING,ELLY VAN GELDEREN,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA
493817,9789027213273,AUD,78.95,PRONUNCIATION FUNDAMENTALS,JOHN BENJAMINS PUBLISHING,TRACEY M. DERWING; MURRAY J. MUNRO,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA
493818,9789027219060,AUD,75.95,COGNITIVE EXPLORATION OF LANGUAGE AND LINGUISTICS,JOHN BENJAMINS PUBLISHING,RENE DIRVEN,JOHN BENJAMINS PUBLISHING,JOHN BENJAMINS PUBLISHING,COINFO,JOHN BENJAMINS PUBLISHING,0.30,3.30,10000,0,AUSTRALIA


In [147]:
sup_list3.drop(columns = ['Publisher_x', 'PublisherLong', 'Publisher_y'], inplace = True)

In [148]:
sup_list3.head()

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region
0,9780000079145,GBP,67.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE
1,9780000002679,GBP,425.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE
2,9780000050090,GBP,17.99,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE
3,9780000003201,GBP,265.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE
4,9780000002402,GBP,306.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE


In [149]:
# 5. Check Suppliers special discounts
sp_disc = pd.read_excel("../../../../Vendors/. Short Discounted/all_short_discount_list.xlsx")

In [150]:
sp_disc.head()

,SUPPLIER,ISBN,DISCOUNT
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


In [151]:
sp_disc.dtypes

SUPPLIER     object
ISBN          int64
DISCOUNT    float64
dtype: object

In [152]:
sp_disc['ISBN'] = sp_disc['ISBN'].astype(str)
sp_disc.dtypes

SUPPLIER     object
ISBN         object
DISCOUNT    float64
dtype: object

In [153]:
for col in ['SUPPLIER', 'ISBN']:
    sp_disc[col] = sp_disc.apply(lambda x: clean_up(x[col]), axis =1)

sp_disc.head()

,SUPPLIER,ISBN,DISCOUNT
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


In [154]:
sup_list4 = pd.merge(sup_list3, sp_disc, how='left', left_on = ['Supplier', 'Isbn'], right_on = ['SUPPLIER', 'ISBN'] )

In [155]:
sup_list4.head()

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT
0,9780000079145,GBP,67.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN
1,9780000002679,GBP,425.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN
2,9780000050090,GBP,17.99,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN
3,9780000003201,GBP,265.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN
4,9780000002402,GBP,306.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN


In [156]:
sup_list4[sup_list4['SUPPLIER']=='ABE']

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT
125,9780781766173,GBP,65.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781766173,0.35
131,9780781768702,GBP,55.00,ACSM'S RESOURCES FOR CLINICAL EXERCISE PHYSIOLOGY,AMERICAN COLLEGE OF SPORTS MEDICINE,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781768702,0.35
137,9780781767538,GBP,48.50,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781767538,0.35
148,9780781774833,GBP,49.99,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781774833,0.35
149,9780781775663,GBP,31.50,STUDY GUIDE FOR PHARMACOLOGY FOR HEALTH PROFES...,RENEE ACOSTA,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781775663,0.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480272,9780393937282,GBP,15.00,DEFINING A NATION,AINSLIE T. EMBREE; MARK C. CARNES,NORTON,ABE,0.30,3.47,10000,0,EUROPE,ABE,9780393937282,-0.05
480284,9780393937275,GBP,17.99,CONFUCIANISM AND THE SUCCESSSION CRISIS OF THE...,DANIEL K. GARDNER; MARK C. CARNES,NORTON,ABE,0.30,3.47,10000,0,EUROPE,ABE,9780393937275,0.25
480297,9780393930047,GBP,49.99,PHYSICS FOR ENGINEERS AND SCIENTISTS,HANS C. OHANIAN; JOHN T. MARKERT,NORTON,ABE,0.30,3.47,10000,0,EUROPE,ABE,9780393930047,0.25
480544,9780393925227,GBP,53.00,ECONOMICS OF THE PUBLIC SECTOR,JOSEPH E. STIGLITZ; JAY K. ROSENGARD,NORTON,ABE,0.30,3.47,10000,0,EUROPE,ABE,9780393925227,0.25


In [157]:
sup_list4.isna().sum()

Isbn                   0
Currency               0
Price                  0
Title                  0
Author                 0
PublisherShort         0
Supplier               0
Discount               0
Shipping Cost          0
max quantity           0
Min                    0
Region                 0
SUPPLIER          492321
ISBN              492321
DISCOUNT          492321
dtype: int64

In [158]:
sup_list4['final_discount'] = np.where(sup_list4['DISCOUNT'].isna(), sup_list4['Discount'],sup_list4['DISCOUNT'])

In [159]:
sup_list4.head()

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT,final_discount
0,9780000079145,GBP,67.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55
1,9780000002679,GBP,425.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55
2,9780000050090,GBP,17.99,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55
3,9780000003201,GBP,265.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55
4,9780000002402,GBP,306.00,N/A,N/A,W KLUWER,ABE,0.55,3.47,10000,0,EUROPE,NaN,NaN,NaN,0.55


In [160]:
sup_list4[sup_list4['Isbn'] == '9780781762021']

,Isbn,Currency,Price,Title,Author,PublisherShort,Supplier,Discount,Shipping Cost,max quantity,Min,Region,SUPPLIER,ISBN,DISCOUNT,final_discount
305247,9780781762021,GBP,57.0,N/A,N/A,LWW,ABE,0.55,3.47,10000,0,EUROPE,ABE,9780781762021,0.35,0.35


In [161]:
sup_list4 = sup_list4[['Isbn','Title', 'Author', 'PublisherShort', 'Supplier','Currency', 'Price', 'final_discount' ,'Shipping Cost', 'max quantity', 'Min', 'Region']]

In [162]:
sup_list4.head()

,Isbn,Title,Author,PublisherShort,Supplier,Currency,Price,final_discount,Shipping Cost,max quantity,Min,Region
0,9780000079145,N/A,N/A,W KLUWER,ABE,GBP,67.00,0.55,3.47,10000,0,EUROPE
1,9780000002679,N/A,N/A,W KLUWER,ABE,GBP,425.00,0.55,3.47,10000,0,EUROPE
2,9780000050090,N/A,N/A,W KLUWER,ABE,GBP,17.99,0.55,3.47,10000,0,EUROPE
3,9780000003201,N/A,N/A,W KLUWER,ABE,GBP,265.00,0.55,3.47,10000,0,EUROPE
4,9780000002402,N/A,N/A,W KLUWER,ABE,GBP,306.00,0.55,3.47,10000,0,EUROPE


In [163]:
sup_list4.columns = map(str.lower, sup_list4.columns)

In [164]:
sup_list4.head()

,isbn,title,author,publishershort,supplier,currency,price,final_discount,shipping cost,max quantity,min,region
0,9780000079145,N/A,N/A,W KLUWER,ABE,GBP,67.00,0.55,3.47,10000,0,EUROPE
1,9780000002679,N/A,N/A,W KLUWER,ABE,GBP,425.00,0.55,3.47,10000,0,EUROPE
2,9780000050090,N/A,N/A,W KLUWER,ABE,GBP,17.99,0.55,3.47,10000,0,EUROPE
3,9780000003201,N/A,N/A,W KLUWER,ABE,GBP,265.00,0.55,3.47,10000,0,EUROPE
4,9780000002402,N/A,N/A,W KLUWER,ABE,GBP,306.00,0.55,3.47,10000,0,EUROPE


In [165]:
# Add western campus
# Shipping Cost = 2
# 0-150CAD 10% off
# 150-200CAD 12% off
# > 200CAD 14%

can_sup = sup_list2[sup_list2['Currency'] == 'CND']
can_sup.columns = map(str.lower, can_sup.columns)
can_sup['supplier'] = 'WC'
can_sup['final_discount'] = np.where(can_sup['price']<150, 0.1, np.where((can_sup['price'] >=150) & (can_sup['price']<200),0.12,0.14))
can_sup['max quantity'] = 10000
can_sup['region'] = 'NORTH AMERICA'
can_sup['shipping cost'] = 2
can_sup['min'] = 0
can_sup.head()

,isbn,currency,price,title,publisher,author,publisherlong,publishershort,supplier,final_discount,max quantity,region,shipping cost,min
1,9780008135904,CND,9.99,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0
6,9780008135935,CND,7.99,COLLINS PORTUGUESE PHRASEBOOK AND DICTIONARY G...,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0
14,9780008136628,CND,28.99,COLLINS WORLD ATLAS: ILLUSTRATED EDITION,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0
29,9780008141790,CND,17.99,COLLINS ENGLISH POCKET DICTIONARY AND THESAURUS,HARPER COLLINS,N/A,HARPER COLLINS,HARPER COLLINS,WC,0.1,10000,NORTH AMERICA,2,0
33,9780008141806,CND,17.99,COLLINS ENGLISH POCKET DICTIONARY,N/A,N/A,N/A,N/A,WC,0.1,10000,NORTH AMERICA,2,0


In [166]:
can_sup = can_sup[['isbn', 'title', 'author', 'publishershort', 'supplier','currency','price','final_discount', 'shipping cost', 'max quantity', 'min', 'region']]
can_sup['currency'] = np.where(can_sup['currency'] == 'CND', 'CAD',0)

In [167]:
can_sup.head()

,isbn,title,author,publishershort,supplier,currency,price,final_discount,shipping cost,max quantity,min,region
1,9780008135904,COLLINS MANDARIN CHINESE PHRASEBOOK AND DICTIO...,N/A,N/A,WC,CAD,9.99,0.1,2,10000,0,NORTH AMERICA
6,9780008135935,COLLINS PORTUGUESE PHRASEBOOK AND DICTIONARY G...,N/A,N/A,WC,CAD,7.99,0.1,2,10000,0,NORTH AMERICA
14,9780008136628,COLLINS WORLD ATLAS: ILLUSTRATED EDITION,N/A,N/A,WC,CAD,28.99,0.1,2,10000,0,NORTH AMERICA
29,9780008141790,COLLINS ENGLISH POCKET DICTIONARY AND THESAURUS,N/A,HARPER COLLINS,WC,CAD,17.99,0.1,2,10000,0,NORTH AMERICA
33,9780008141806,COLLINS ENGLISH POCKET DICTIONARY,N/A,N/A,WC,CAD,17.99,0.1,2,10000,0,NORTH AMERICA


In [168]:
sup_list5 = sup_list4.append(can_sup, ignore_index= True)
sup_list5.head()

,isbn,title,author,publishershort,supplier,currency,price,final_discount,shipping cost,max quantity,min,region
0,9780000079145,N/A,N/A,W KLUWER,ABE,GBP,67.00,0.55,3.47,10000,0,EUROPE
1,9780000002679,N/A,N/A,W KLUWER,ABE,GBP,425.00,0.55,3.47,10000,0,EUROPE
2,9780000050090,N/A,N/A,W KLUWER,ABE,GBP,17.99,0.55,3.47,10000,0,EUROPE
3,9780000003201,N/A,N/A,W KLUWER,ABE,GBP,265.00,0.55,3.47,10000,0,EUROPE
4,9780000002402,N/A,N/A,W KLUWER,ABE,GBP,306.00,0.55,3.47,10000,0,EUROPE


In [169]:
sup_list5['currency'].unique()

array(['GBP', 'AUD', 'USD', 'CAD'], dtype=object)

In [170]:
# Exchange Rate
#We add the Exchange rate column
exchange_rate = """
SELECT *
FROM dbo.XChange
"""
exch_df = pd.read_sql(exchange_rate,prouc_db)


In [171]:
exch_df

,id,name,rate,updatedon
0,2,AUD,0.617816,2020-04-07 08:00:00.757
1,3,NZD,0.598695,2020-04-07 08:00:00.757
2,4,GBP,1.231148,2020-04-07 08:00:00.757
3,5,CAD,0.713323,2020-04-07 08:00:00.757
4,6,EUR,1.087406,2020-04-07 08:00:00.757
5,8,SGD,0.701688,2020-04-07 08:00:00.757
6,9,HKD,0.128986,2020-04-07 08:00:00.757
7,1,USD,1.000000,2020-04-07 08:00:00.757
8,10,TWD,0.033135,2020-04-07 08:00:00.757
9,11,CHF,1.022320,2020-04-07 08:00:00.757


In [172]:
sup_list6 = pd.merge(sup_list5, exch_df, how='left', left_on = 'currency', right_on = 'name' )
sup_list6['invoice_fee'] = np.where(sup_list6['supplier'] == 'COINFO', (-1*sup_list6['price']*(1-sup_list6['final_discount']))/11  , 0)
sup_list6['lp_fee'] = 0
sup_list6.rename(columns={'publishershort': 'pub', 'final_discount': 'discount'}, inplace = True)


In [173]:
sup_list6 = sup_list6[['isbn', 'title', 'author', 'pub', 'supplier','currency','rate','price','discount','invoice_fee','lp_fee','shipping cost', 'max quantity', 'min', 'region']]
sup_list6.head()

,isbn,title,author,pub,supplier,currency,rate,price,discount,invoice_fee,lp_fee,shipping cost,max quantity,min,region
0,9780000079145,N/A,N/A,W KLUWER,ABE,GBP,1.231148,67.00,0.55,0.0,0,3.47,10000,0,EUROPE
1,9780000002679,N/A,N/A,W KLUWER,ABE,GBP,1.231148,425.00,0.55,0.0,0,3.47,10000,0,EUROPE
2,9780000050090,N/A,N/A,W KLUWER,ABE,GBP,1.231148,17.99,0.55,0.0,0,3.47,10000,0,EUROPE
3,9780000003201,N/A,N/A,W KLUWER,ABE,GBP,1.231148,265.00,0.55,0.0,0,3.47,10000,0,EUROPE
4,9780000002402,N/A,N/A,W KLUWER,ABE,GBP,1.231148,306.00,0.55,0.0,0,3.47,10000,0,EUROPE


In [174]:
# sup_list6.to_csv('suppliers_excel.csv', index= False)

In [175]:
sup_list6[sup_list6['supplier'] == 'COINFO']

,isbn,title,author,pub,supplier,currency,rate,price,discount,invoice_fee,lp_fee,shipping cost,max quantity,min,region
198592,9780060419257,THE MICROBIOLOGY COLORING BOOK,LAWRENCE M. ELSON; I. EDWARD ALCAMO,PEARSON,COINFO,AUD,0.617816,31.95,0.4,-1.742727,0,3.3,10000,0,AUSTRALIA
198593,9780020130857,ELEMENTS OF TECHNICAL WRITING,GARY BLAKE; ROBERT W. BLY,PEARSON,COINFO,AUD,0.617816,14.95,0.4,-0.815455,0,3.3,10000,0,AUSTRALIA
198594,9780023285219,ENGINEERING ELECTROMAGNETICS,KENNETH R. DEMAREST,PEARSON,COINFO,AUD,0.617816,299.95,0.4,-16.360909,0,3.3,10000,0,AUSTRALIA
198595,9780023364501,PRINCIPLES AND APPLICATIONS OF GEOCHEMISTRY,GUNTER FAURE,PEARSON,COINFO,AUD,0.617816,240.95,0.4,-13.142727,0,3.3,10000,0,AUSTRALIA
198596,9780023464508,THE CONSOLATION OF PHILOSOPHY,RICHARD H. GREEN,PEARSON,COINFO,AUD,0.617816,44.95,0.4,-2.451818,0,3.3,10000,0,AUSTRALIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493823,9789027252722,"PIDGINS, CREOLES AND MIXED LANGUAGES: AN INTRO...",VIVEKA VELUPILLAI,JOHN BENJAMINS PUBLISHING,COINFO,AUD,0.617816,78.95,0.3,-5.024091,0,3.3,10000,0,AUSTRALIA
493824,9789027212092,HISTORY OF THE ENGLISH LANGUAGE,ELLY VAN GELDEREN,JOHN BENJAMINS PUBLISHING,COINFO,AUD,0.617816,79.95,0.3,-5.087727,0,3.3,10000,0,AUSTRALIA
493825,9789027213273,PRONUNCIATION FUNDAMENTALS,TRACEY M. DERWING; MURRAY J. MUNRO,JOHN BENJAMINS PUBLISHING,COINFO,AUD,0.617816,78.95,0.3,-5.024091,0,3.3,10000,0,AUSTRALIA
493826,9789027219060,COGNITIVE EXPLORATION OF LANGUAGE AND LINGUISTICS,RENE DIRVEN,JOHN BENJAMINS PUBLISHING,COINFO,AUD,0.617816,75.95,0.3,-4.833182,0,3.3,10000,0,AUSTRALIA
